In [ ]:
pip install opencv-python

In [ ]:
pip install matplotlib

In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import RandomState
import pandas as pd

# Generate Point Object file

In [ ]:
pip install open3d

In [ ]:
import numpy as np
import open3d as o3d

def use_o3d(pts, write_text):
    pcd = o3d.geometry.PointCloud()

    # the method Vector3dVector() will convert numpy array of shape (n, 3) to Open3D format.
    # see http://www.open3d.org/docs/release/python_api/open3d.utility.Vector3dVector.html#open3d.utility.Vector3dVector
    pcd.points = o3d.utility.Vector3dVector(pts)

    # http://www.open3d.org/docs/release/python_api/open3d.io.write_point_cloud.html#open3d.io.write_point_cloud
    o3d.io.write_point_cloud("points.ply", pcd, write_ascii=write_text)

    # read ply file
    pcd = o3d.io.read_point_cloud('points.ply')

    # visualize
    o3d.visualization.draw_geometries([pcd])

In [ ]:
# use_o3d(points,True)

## Functions

In [ ]:
def calculate_action_probability(state):
  exponentiated_potential = np.exp(-theta[state[0]][state[1]])
  action_probability = 1/(exponentiated_potential+1)
  return action_probability

In [ ]:
def transition(state, action_probability, boundary):
  rand = np.random.random()

  if state[0] > boundary:
    action = 0
    action_probability = 1
  elif state[0] < -boundary:
    action = 1
    action_probability = 1
  else:
    if rand < action_probability:
      action = 1
      action_probability = action_probability
    else:
      action = 0
      action_probability = 1-action_probability
  return action, action_probability

In [ ]:
def update_state(state, action):
  state[0] += 2*action - 1
  state[1] += 1
  return state

In [ ]:
def generate_reward(state, controlled_state_end, controlled_time):

  #controlled_state = 20
  #controlled_time = 20
  controlled_multiplier = 2

  if state[0] < 0:
    reward = -1 * abs(state[0])
  else:
    reward = 0
  if state[1] == controlled_time:
    reward -=  controlled_multiplier * abs(controlled_state_end - state[0])
  return reward

In [ ]:
def calculate_eligibility(state, action, action_probability):
  if action == 1:
    eligibility = 1 - action_probability
  else:
    eligibility = -action_probability
  return eligibility

In [ ]:
def update_theta(state, next_state, reward, learning_rate, eligibility): #actor-critic
  past_value = find_value(state)
  current_value = find_value(next_state)
  td_error = current_value + reward - past_value
  theta[state[0]][state[1]] += learning_rate * td_error * eligibility
  return theta

In [ ]:
def find_value(state):
  return value_table[state[0]][state[1]]

In [ ]:
def update_value(state, next_state, reward, learning_rate_beta): #actor-critic
  past_value = find_value(state)
  current_value = find_value(next_state)
  td_error = current_value + reward - past_value
  value_table[state[0]][state[1]] += learning_rate_beta * td_error
  return value_table

In [ ]:
def generate_samples(N, box_boundary, controlled_state_start, controlled_state_end, controlled_time):
  trajectories = []
  average_return = 0
  for i in range(0, N):
    state = [controlled_state_start,0]
    trajectory = [state.copy()]
    trajectory_reward = 0
    for i in range (0,controlled_time):
      action_probability = calculate_action_probability(state.copy())
      action, action_probability = transition(state.copy(), action_probability, box_boundary)
      next_state = update_state(state.copy(), action)
      reward = generate_reward(next_state.copy(), controlled_state_end, controlled_time)
      trajectory_reward += reward
      state = next_state
      trajectory.append(state.copy())
    trajectories.append(trajectory.copy())
    average_return += (trajectory_reward - average_return)/(i+1)
  return trajectories, average_return

## Gaussian dynamics

In [ ]:
def gaussian_derivative(x,sigma):
  Gaussian = np.exp(-x**2/sigma**2)*x/sigma**2
  return Gaussian

In [ ]:
def dynamics(a,zlim,steps,sigma):
  N = 2*a
  z = np.arange(-zlim,zlim,steps)
  #z = ((z+zlim)/steps).astype(int)
  f = gaussian_derivative(z,sigma)

  pminus = (f+a)/N
  pplus = (-f+a)/N

  pminus[0] = 1
  pplus[0] = 0.0001

  pminus[-1] = 0.0001
  pplus[-1] = 1

  return z, pminus, pplus

In [ ]:
def kl_regularisation(state, action, action_probability):

  z, pminus, pplus = dynamics(0.5, 10, 1/50, 1)

  #plt.plot(z,pminus)
  #plt.plot(z,pplus)
  
  if action == 1:
    prob = pplus[state[0]]
  else:
    prob = pminus[state[0]]
  #print(prob)

  return -math.log(action_probability/prob)

In [ ]:
z, p1, p2 = dynamics(0.5, 10, 1/50, 1)

plt.plot(z,p1)
plt.plot(z,p2)

plt.xlabel('z')
plt.ylabel('P')

##Actor-Critic algorithm

In [ ]:
def actor_critic(N, avg, box_boundary, controlled_state_start, controlled_state_end, controlled_time):
  trajectories = []
  trajectory_rewards = [] #stores trajectory rewards
  running_average_return = [] #store running average return
  for i in range(0, N):
    state = [controlled_state_start,0]
    trajectory = [state.copy()]
    actions = []
    rewards = []
    eligibilities = []
    action_probabilities = []
    trajectory_reward = 0
    for i in range (0,controlled_time):
      action_probability = calculate_action_probability(state.copy())

      action, action_probability = transition(state.copy(), action_probability, box_boundary)
      actions.append(action) #store actions
      action_probabilities.append(action_probability) #store action probabilities

      eligibility = calculate_eligibility(state.copy(), action, action_probability)
      eligibilities.append(eligibility) #store eligibilities
      
      next_state = update_state(state.copy(), action)
  
      reward = generate_reward(next_state.copy(), controlled_state_end, controlled_time)#+ kl_regularisation(state, action, action_probability)
      rewards.append(reward) #store rewards

      trajectory_reward += reward 

      #update the value for past state
      update_value(state, next_state, reward, learning_rate_beta)

      #update theta
      update_theta(state, next_state, reward, learning_rate, eligibility)

      state = next_state #set next state as state
      trajectory.append(state.copy()) #add next state to trajectory

    trajectories.append(trajectory.copy()) #append trajectory to global list
    trajectory_rewards.append(trajectory_reward) # append reward for trajectory

    #average_return
    avg += return_learning_rate * (trajectory_reward - avg)
    running_average_return.append(avg)

  return trajectory_rewards, running_average_return

##Untrained Policy

Set variables

In [ ]:
T = 20  # Longer time = more possible combinations so better encryption
start_state = 20
end_state = 20

dimensions = (2*T+1,T+1)
theta = np.zeros(dimensions)

Nbody = 304  #1166
xlimit = 700
ylimit = 400
zlimit = 400

xuntrained, xavg = generate_samples(Nbody, xlimit, start_state, end_state, T)

In [ ]:
theta = np.zeros(dimensions)
yuntrained, yavg = generate_samples(Nbody, ylimit, start_state, end_state, T)

In [ ]:
theta = np.zeros(dimensions)
zuntrained, zavg = generate_samples(Nbody, zlimit, start_state, end_state, T)

In [ ]:
min_y = np.min(np.array(xuntrained)[:,:,0]) - 1
max_y = np.max(np.array(xuntrained)[:,:,0]) + 1

plt.plot(np.array(xuntrained)[:,:,0].T, c = 'k', alpha = 0.2)
plt.scatter([T], [end_state], c = 'k', marker = 'o', s = 100)
plt.plot([-1, T+1], [0, 0], lw = 2, c = 'r', ls = '--', alpha = 0.3)
plt.fill_between([-1, T+1], [0, 0], [min_y, min_y], color = 'r', alpha = 0.1)
plt.xlim(-1, T+1)
plt.ylim(min_y)
plt.xlabel("Time", fontsize=16)
plt.ylabel("Position", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=14)
plt.show()

print("Average return of initial policy:", xavg)

##Train actor-critic

Set variables

In [ ]:
#from locale import T_FMT
learning_rate = 0.15
learning_rate_beta = 0.6
return_learning_rate = 0.1
N = 10000

In [ ]:
theta = np.zeros(dimensions)
value_table = np.zeros(dimensions)

xac_rewards, xac_running_return = actor_critic(N, xavg, xlimit, start_state, end_state, T)
xtrained, xtrained_avg = generate_samples(Nbody, xlimit, start_state, end_state, T)

In [ ]:
theta = np.zeros(dimensions)
value_table = np.zeros(dimensions)

yac_rewards, yac_running_return = actor_critic(N, yavg, ylimit, start_state, end_state, T)
ytrained, ytrained_avg = generate_samples(Nbody, ylimit, start_state, end_state, T)

In [ ]:
theta = np.zeros(dimensions)
value_table = np.zeros(dimensions)

zac_rewards, zac_running_return = actor_critic(N, zavg, zlimit, start_state, end_state, T)
ztrained, ztrained_avg = generate_samples(Nbody, zlimit, start_state, end_state, T)

Plot episodic returns

In [ ]:
plt.plot(xac_rewards, c = 'b', alpha = 1)
plt.xlabel("Episode")
plt.ylabel("Episodic returns")

Plot running returns

In [ ]:
plt.plot(xac_running_return, c = 'b', alpha = 1)
plt.xlabel("Episode")
plt.ylabel("Running return")
plt.show()

Plot optimised policy trajectories with updated theta, plot 30 trajectories and compare to untrained policy

In [ ]:
min_y = np.min(np.array(xtrained)[:,:,0]) - 1
max_y = np.max(np.array(xtrained)[:,:,0]) + 1

plt.plot(np.array(xuntrained)[:,:,0].T, c = 'k', alpha = 0.2)
plt.plot(np.array(xtrained)[:,:,0].T, c = 'b', alpha = 0.2)
plt.plot([-1, T+1], [0, 0], lw = 2, c = 'r', ls = '--', alpha = 0.3)
plt.scatter([T], [end_state], c = 'k', marker = 'o', s = 100)
plt.fill_between([-1, T+1], [0, 0], [-10, -10], color = 'r', alpha = 0.1)
plt.xlim(-1, T+1)
plt.ylim(-10)
plt.xlabel("Time")
plt.ylabel("Position")
# plt.xticks(fontsize=16)
# plt.yticks(fontsize=16)
plt.show()

print("Average return x:", xtrained_avg)
print("Average return y:", ytrained_avg)
print("Average return z:", ztrained_avg)

# 2D

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
from numpy.random import randn
from time import sleep

for t in range(0,T+1,T//10):

  print(t)
  plt.scatter(np.array(xtrained)[:,t,0].T,np.array(ytrained)[:,t,0].T, alpha = 0.1)
  plt.xlim(-5, 75)
  plt.ylim(-5, 45)
  plt.show()

  sleep(10/T)
  clear_output(wait=True)
  
  

In [ ]:
np.array(xtrained)[:,:,0]

In [ ]:
np.array(ytrained)[:,:,0]

# 3D

In [ ]:
import matplotlib.animation as animation

In [ ]:
from matplotlib import rc
rc('animation', html='jshtml')

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

t = 0

def frame(w):
  ax.clear()
  global t

  x=(np.array(xtrained)[:,t,0].T-20)
  y=(np.array(ytrained)[:,t,0].T-20)
  z=np.array(ztrained)[:,t,0].T-20

  ax.set_xlim3d(-20, 20)
  ax.set_ylim3d(-20, 20)
  ax.set_zlim3d(-20, 20)
  plot = ax.scatter3D(x,y,z, alpha = 0.5)

  t+=T//10

  return plot
plt.close()

anim = animation.FuncAnimation(fig, frame, frames=10, blit=False, repeat=True)
anim

# Ordered trajectories

In [ ]:
%cd C:\Users\brinp\Documents\Jupyter\Quantum Machine Learning\Quantum machine encryption

In [ ]:
ordered_orig = pd.read_excel('orderR.xlsx')
random_orig = pd.read_excel('SameRmussel.xlsx')
random_orig['Center Z'] = np.round(20*np.random.rand(304),4)
random_orig

In [ ]:
plt.scatter(ordered_orig['Center X'], ordered_orig['Center Y'], s = ordered_orig['Radius'])
plt.scatter(random_orig['Center X'], random_orig['Center Y'], s = random_orig['Radius'])

In [ ]:
order_xMatrix = np.vstack([ordered_orig['Center X']]*(T+1))
order_yMatrix = np.vstack([ordered_orig['Center Y']]*(T+1))
order_zMatrix = np.vstack([ordered_orig['Center Y']]*(T+1))

order_xtrained = np.transpose(order_xMatrix) + np.array(xtrained)[:,:,0] -20
order_ytrained = np.transpose(order_yMatrix) + np.array(ytrained)[:,:,0] -20
order_ztrained = np.transpose(order_zMatrix) + np.array(ztrained)[:,:,0] -20

In [ ]:
t = 0

plt.scatter(np.array(order_xtrained)[:,t].T,np.array(order_ytrained)[:,t].T)

In [ ]:
for t in range(0,T+1,T//10):

  print(t)
  plt.scatter(np.array(order_xtrained)[:,t].T,np.array(order_ytrained)[:,t].T)
  plt.xlim(-5, 75)
  plt.ylim(-5, 45)
  plt.show()

  sleep(10/T)
  clear_output(wait=True)

In [ ]:
# 2D distances

In [ ]:
most_random = 0*np.ones([304,21])

distance = ((most_random - np.transpose(order_xMatrix))**2 + (most_random - np.transpose(order_yMatrix))**2)**0.5

distance_sum = np.sum(distance**2, axis = 0)
order_amount = (distance_sum / (Nbody**0.5 *(40**2+70**2)))**0.5  # amount of order
np.round(order_amount,3)/2.318

In [ ]:
distance = ((order_xtrained - np.transpose(order_xMatrix))**2 + (order_ytrained - np.transpose(order_yMatrix))**2)**0.5

distance_sum = np.sum(distance**2, axis = 0)
order_amount = (distance_sum / (Nbody**0.5 *(40**2+70**2)))**0.5  # amount of order
np.round(order_amount,3)/2.318

In [ ]:
pd.DataFrame(order_xtrained).to_excel('x.xlsx')
pd.DataFrame(order_ytrained).to_excel('y.xlsx')
pd.DataFrame(order_amount).to_excel('order_amount.xlsx')

# 3D

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

t = 0

def frame(w):
  ax.clear()
  global t

  x=np.array(order_xtrained)[:,t].T
  y=np.array(order_ytrained)[:,t].T
  z=np.array(order_ztrained)[:,t].T

  ax.set_xlim3d(-5, 75)
  ax.set_ylim3d(-5, 45)
  ax.set_zlim3d(-5, 45)
  plot = ax.scatter3D(x,y,z, alpha = 0.5)

  t+=T//10

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=10, blit=False, repeat=True)
anim

# Generator

## 2D

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

order = 1

# specify params
n = Nbody
shape = np.array([40, 70])
sensitivity = 0.8 # 0 means no movement, 1 means max distance is init_dist

# compute grid shape based on number of points
width_ratio = shape[1] / shape[0]
num_y = np.int32(np.sqrt(n / width_ratio)) + 1
num_x = np.int32(n / num_y) + 1

# create regularly spaced neurons
x = np.linspace(0., shape[1]-1, num_x, dtype=np.float32)
y = np.linspace(0., shape[0]-1, num_y, dtype=np.float32)
coords = np.stack(np.meshgrid(x, y), -1).reshape(-1,2)

# compute spacing
init_dist = np.min((x[1]-x[0], y[1]-y[0]))
min_dist = init_dist * (1 - sensitivity)

assert init_dist >= min_dist
print(min_dist)

# perturb points
max_movement = order*(init_dist - min_dist)/2
noise = np.random.uniform(
    low=-max_movement,
    high=max_movement,
    size=(len(coords), 2))
coords += noise

# plot
plt.figure(figsize=(5*width_ratio,5))
plt.scatter(coords[:,0], coords[:,1], s=3)
plt.show()

In [ ]:
def generator2D(xdata, ydata, order):
    
    # https://stackoverflow.com/questions/66238749/how-to-find-the-closest-coordinate-from-a-list-of-points
    
  amount = len(xdata)

  x_random = np.random.uniform(low=0, high=70, size=(amount))
  y_random = np.random.uniform(low=0, high=40, size=(amount))
    
  most_random = np.random.random()*np.ones([amount,21])
  distance_rand = ((most_random - np.transpose(order_xMatrix))**2 + (most_random - np.transpose(order_yMatrix))**2)**0.5
  distance_sum_rand = np.sum(distance_rand**2, axis = 0)
  order_amount = (distance_sum_rand / (Nbody**0.5 *((max(xdata)-min(xdata))**2+(max(ydata)-min(ydata))**2)))**0.5  # amount of order
  most_random_factor = np.round(order_amount[0],3)
    
  
  distance_sum = amount*order*most_random_factor*((max(xdata)-min(xdata))**2+(max(ydata)-min(ydata))**2)**0.5  # amount of order
  
  av_distance = distance_sum/amount

  distance = av_distance*np.random.rand(amount)
  
  theta = 2*np.pi *np.random.rand(amount)

  xdist = abs(distance*abs(np.transpose(np.cos(theta))))
  ydist = abs(distance*abs(np.transpose(np.sin(theta))))


  genx = (most_random_factor - order*most_random_factor)*(xdata + xdist)/most_random_factor
  geny = (most_random_factor - order*most_random_factor)*(ydata + ydist)/most_random_factor
  
  return genx, geny

In [ ]:
order = 0.5

xg, yg = generator2D(ordered_orig['Center X'], ordered_orig['Center Y'],order)

plt.scatter(xg, yg)

#pd.DataFrame([xg, yg]).to_excel('2DGen/2Dgen_order_'+str(order)+'.xlsx')

## 3D

In [ ]:
def generator3D(xdata, ydata, zdata, order):
  
  amount = len(xdata)
  most_random = 0*np.ones([amount,21])
  distance_rand = ((most_random - np.transpose(order_xMatrix))**2 + (most_random - np.transpose(order_yMatrix))**2+ (most_random - np.transpose(order_zMatrix))**2)**0.5
  distance_sum_rand = np.sum(distance_rand**2, axis = 0)
  order_amount = (distance_sum_rand / (Nbody**0.5 *((max(xdata)-min(xdata))**2+(max(ydata)-min(ydata))**2+(max(zdata)-min(zdata))**2)))**0.5  # amount of order
  most_random_factor = np.round(order_amount[0],3)
  
  distance_sum = amount*order*most_random_factor*((max(xdata)-min(xdata))**2+(max(ydata)-min(ydata))**2+(max(zdata)-min(zdata))**2)**0.5  # amount of order
  
  av_distance = distance_sum/amount

  distance = av_distance*np.random.rand(amount)
  
  theta = 2*np.pi *np.random.rand(amount)
  phi = 2*np.pi *np.random.rand(amount) - np.pi

  # spherical coordinates
  xdist = abs(distance*abs(np.transpose(np.cos(theta)*np.sin(phi))))
  ydist = abs(distance*abs(np.transpose(np.sin(theta)*np.sin(phi))))
  zdist = abs(distance*abs(np.transpose(np.cos(phi))))

  genx = (most_random_factor - order*most_random_factor)*(xdata + xdist)/most_random_factor
  geny = (most_random_factor - order*most_random_factor)*(ydata + ydist)/most_random_factor
  genz = (most_random_factor - order*most_random_factor)*(zdata + zdist)/most_random_factor
  
  return genx, geny, genz

In [ ]:
order = 0

xg, yg, zg = generator3D(ordered_orig['Center X'],ordered_orig['Center Y'],ordered_orig['Center Y'],order)

fig = plt.figure()
ax = plt.axes(projection='3d')

ax.scatter3D(xg,yg,zg)

# 3D diffusion to ordered

In [ ]:
def fibonacci_hyperbola(samples, amplitude, xc, yc, zc):

    points = []
    phi = np.pi * (3. - np.sqrt(5.))  # golden angle in radians

    for i in range(samples):
        y = amplitude[i]*(1 - (i / float(samples - 1)) * 2)  # y goes from 1 to -1
        radius = np.sqrt(y**2 *amplitude[i]**2 - y)  # radius at y

        theta = phi * i  # golden angle increment

        x = np.cos(theta) * radius
        z = np.sin(theta) * radius 

        points.append((x+xc, y+yc, z+zc))

    return points

In [ ]:
def fibonacci(samples, amplitude, xc, yc, zc):

    points = []
    phi = np.pi * (3. - np.sqrt(5.))  # golden angle in radians

    for i in range(samples):
        y = amplitude[i]*(1 - (i / float(samples - 1)) * 2)  # y goes from 1 to -1
        radius = (y**-1 * amplitude[i]**-2 - y)  # radius at y

        theta = phi * i  # golden angle increment

        x = np.cos(theta) * radius
        z = np.sin(theta) * radius 

        points.append((x+xc, y+yc, z+zc))

    return points

## Sphere creator

In [ ]:
def fibonacci_sphere(samples, amplitude, xc, yc, zc):

    points = []
    phi = np.pi * (3. - np.sqrt(5.))  # golden angle in radians

    for i in range(samples):
        y = amplitude[i]*(1 - (i / float(samples - 1)) * 2)  # y goes from 1 to -1
        radius = np.sqrt(amplitude[i]**2 - y * y)  # radius at y

        theta = phi * i  # golden angle increment

        x = np.cos(theta) * radius
        z = np.sin(theta) * radius

        points.append((x+xc, y+yc, z+zc))

    return points

Cube Creator

In [ ]:
def fibonacci_cube(samples, amplitude, xc, yc, zc):

    points = []
    phi = np.pi * (3. - np.sqrt(5.))  # golden angle in radians

    for i in range(samples):
        y = amplitude[i]*(1 - (i / float(samples - 1)) * 2)  # y goes from 1 to -1
        radius = np.sqrt(amplitude[i]**2 - y * y)  # radius at y

        theta = phi * i  # golden angle increment

        x = np.cos(theta) * radius
        z = np.sin(theta) * radius

        points.append((x+xc, y+yc, z+zc))

    return points

## Quantum Encryption

In [ ]:
# ata

lamb = 1E10
Nbody = 500

xyz = np.linspace(-1,1,Nbody)

A = 20*np.cos(2*np.pi/lamb * xyz)

points = fibonacci(Nbody,A,20,20,20)

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(np.transpose(points)[0],np.transpose(points)[1],np.transpose(points)[2])

In [ ]:
# Data encrypted by Quantum

fig = plt.figure()
ax = plt.axes(projection='3d')

lamb = 1E-10
xyz = np.linspace(-1,1,Nbody)

def frame(w):
  ax.clear()
  global lamb

  A = 1*np.cos(2*np.pi/lamb * xyz)

  points = fibonacci_sphere(Nbody,A,20,20,20)

  plot = ax.scatter3D(np.transpose(points)[0],np.transpose(points)[1],np.transpose(points)[2])
  plot = ax.plot3D(np.transpose(points)[0],np.transpose(points)[1],np.transpose(points)[2],color='r',alpha=0.4)
  #ax.axis(xmin=0,xmax=50)
  #ax.axis(ymin=0,ymax=50)
  #ax.set_zlim(0,50)

  lamb*=1.2

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=400, blit=False, repeat=True)
anim

In [ ]:
# Data encrypted by Quantum

fig = plt.figure()
ax = plt.axes(projection='3d')

lamb = 1E-10
xyz = np.linspace(-1,1,Nbody)

def frame(w):
  ax.clear()
  global lamb

  A = 1*np.cos(2*np.pi/lamb * xyz)

  B = 1*np.sin(2*np.pi/lamb * xyz)

  points = fibonacci_sphere(Nbody,A,20,20,20) + fibonacci_sphere(Nbody,B,22,22,20) 

  plot = ax.scatter3D(np.transpose(points)[0],np.transpose(points)[1],np.transpose(points)[2])
  plot = ax.plot3D(np.transpose(points)[0],np.transpose(points)[1],np.transpose(points)[2],color='r',alpha=0.4)
  #ax.axis(xmin=0,xmax=50)
  #ax.axis(ymin=0,ymax=50)
  #ax.set_zlim(0,50)

  lamb*=1.2

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=400, blit=False, repeat=True)
anim

In [ ]:
# Data encrypted by Quantum

fig = plt.figure()
ax = plt.axes(projection='3d')

lamb = 1E-10
dsdt = 0
a = 0
t = 0
u = 0.01

xyz = np.linspace(-1,1,Nbody)

#animation.embed_limit = 22076560

def frame(w):
  ax.clear()
  global lamb
  global dsdt
  global u
  global a
  global t

  A = 1*np.cos(2*np.pi/lamb * xyz)

  B = 1*np.sin(2*np.pi/lamb * xyz)

  C = 1*np.tan(2*np.pi/lamb * xyz)

  points = fibonacci_sphere(Nbody,A,21,21,20) + fibonacci_sphere(Nbody,B,22,22,18)

  dsdt += (u+a*t) # velocity

  plot = ax.scatter3D(np.transpose(points)[0]+dsdt,np.transpose(points)[1]+dsdt,np.transpose(points)[2]+dsdt)
  plot = ax.plot3D(np.transpose(points)[0]+dsdt,np.transpose(points)[1]+dsdt,np.transpose(points)[2]+dsdt,color='b',alpha=0.4)
  ax.axis(xmin=20,xmax=24)
  ax.axis(ymin=20,ymax=24)
  ax.set_zlim(19,22)
  #fig.set_facecolor('black')
  #ax.set_facecolor('black') 
  #ax.grid(False) 
  #ax.w_xaxis.pane.fill = False
  #ax.w_yaxis.pane.fill = False
  #ax.w_zaxis.pane.fill = False

  lamb*=1.2
  t+=1

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=200, blit=False, repeat=True)
anim

In [ ]:
# Data encrypted by Quantum

fig = plt.figure()
ax = plt.axes(projection='3d')

lamb = 1E-10
dsdt = [0,0,0]
a = 0
t = 0
u = 0.2

xyz = np.linspace(-1,1,Nbody)

#animation.embed_limit = 22076560

def frame(w):
  ax.clear()
  global lamb
  global dsdt
  global u
  global a
  global t

  A = 1*np.cos(2*np.pi/lamb * xyz)

  B = 1*np.sin(2*np.pi/lamb * xyz)

  C = 1*np.tan(2*np.pi/lamb * xyz)

  D = 0.5*np.arctan(2*np.pi/lamb * xyz) # gravity candidate

  points = np.array(fibonacci_sphere(Nbody,D,21,21,20)) * (np.array([D,D,D]).T)

  dsdt[0] += (u+a*t) # x velocity
  dsdt[1] += 0 # y velocity
  dsdt[2] += 0 # z velocity

  plot = ax.scatter3D(np.transpose(points)[0]+dsdt[0],np.transpose(points)[1]+dsdt[1],np.transpose(points)[2]+dsdt[2])
  plot = ax.plot3D(np.transpose(points)[0]+dsdt[0],np.transpose(points)[1]+dsdt[1],np.transpose(points)[2]+dsdt[2],color='b',alpha=0.4)
  #ax.axis(xmin=-30,xmax=30)
  #ax.axis(ymin=-30,ymax=30)
  #ax.set_zlim(-30,30)

  lamb*=1.2
  t+=1

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=200, blit=False, repeat=True)
anim

In [ ]:
# Data encrypted by Quantum

fig = plt.figure()
ax = plt.axes(projection='3d')

lamb = 1E-10
dsdt = [0,0,0]
a = 0
t = 0
u = 0.2

xyz = np.linspace(-1,1,Nbody)

#animation.embed_limit = 22076560

def frame(w):
  ax.clear()
  global lamb
  global dsdt
  global u
  global a
  global t

  A = 1*np.cos(2*np.pi/lamb * xyz)

  B = 1*np.sin(2*np.pi/lamb * xyz)

  C = 1*np.tan(2*np.pi/lamb * xyz)

  D = 0.5*np.arctan(2*np.pi/lamb * xyz) # gravity candidate

  points = np.array(fibonacci_sphere(Nbody,A,21,21,20)) * (np.array([B,xyz,xyz]).T)

  dsdt[0] += (u+a*t) # x velocity
  dsdt[1] += 0 # y velocity
  dsdt[2] += 0 # z velocity

  plot = ax.scatter3D(np.transpose(points)[0]+dsdt[0],np.transpose(points)[1]+dsdt[1],np.transpose(points)[2]+dsdt[2])
  plot = ax.plot3D(np.transpose(points)[0]+dsdt[0],np.transpose(points)[1]+dsdt[1],np.transpose(points)[2]+dsdt[2],color='b',alpha=0.4)
  #ax.axis(xmin=-30,xmax=30)
  #ax.axis(ymin=-30,ymax=30)
  #ax.set_zlim(-30,30)

  lamb*=1.05  #1.2, #1.05
  t+=1

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=200, blit=False, repeat=True)
anim

In [ ]:
# Data encrypted by Quantum

fig = plt.figure()
ax = plt.axes(projection='3d')

lamb = 1E-10
dsdt = [0,0,0]
a = 0
t = 0
u = 0.2

xyz = np.linspace(-1,1,Nbody)

#animation.embed_limit = 22076560

def frame(w):
  ax.clear()
  global lamb
  global dsdt
  global u
  global a
  global t

  A = 1*np.cos(2*np.pi/lamb * xyz)

  B = 1*np.sin(2*np.pi/lamb * xyz)

  C = 1*np.tan(2*np.pi/lamb * xyz)

  D = 0.5*np.arctan(2*np.pi/lamb * xyz) # gravity candidate

  points = np.array(fibonacci_sphere(Nbody,B,21,21,20)) * (np.array([B,A,A]).T)

  dsdt[0] += (u+a*t) # x velocity
  dsdt[1] += 0 # y velocity
  dsdt[2] += 0 # z velocity

  plot = ax.scatter3D(np.transpose(points)[0]+dsdt[0],np.transpose(points)[1]+dsdt[1],np.transpose(points)[2]+dsdt[2])
  plot = ax.plot3D(np.transpose(points)[0]+dsdt[0],np.transpose(points)[1]+dsdt[1],np.transpose(points)[2]+dsdt[2],color='b',alpha=0.4)
  #ax.axis(xmin=-30,xmax=30)
  #ax.axis(ymin=-30,ymax=30)
  #ax.set_zlim(-30,30)

  lamb*=1.2
  t+=1

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=200, blit=False, repeat=True)
anim

In [ ]:
# Data encrypted by Quantum

fig = plt.figure()
ax = plt.axes(projection='3d')

lamb = 1E-10
dsdt = 0
a = 0
t = 0
u = -0.1

xyz = np.linspace(-1,1,Nbody)

#animation.embed_limit = 22076560

def frame(w):
  ax.clear()
  global lamb
  global dsdt
  global u
  global a
  global t

  A = 1*np.cos(2*np.pi/lamb * xyz)

  B = 1*np.sin(2*np.pi/lamb * xyz)

  C = 1*np.tan(2*np.pi/lamb * xyz)

  D = 0.5*np.arctan(2*np.pi/lamb * xyz) # gravity candidate

  points = np.array(fibonacci_sphere(Nbody,D,21,21,20)) * (np.array([D,D,D]).T)**2

  dsdt += (u+a*t) # velocity

  plot = ax.scatter3D(np.transpose(points)[0]+dsdt,np.transpose(points)[1]+dsdt,np.transpose(points)[2]+dsdt)
  plot = ax.plot3D(np.transpose(points)[0]+dsdt,np.transpose(points)[1]+dsdt,np.transpose(points)[2]+dsdt,color='b',alpha=0.4)
  ax.axis(xmin=-30,xmax=30)
  ax.axis(ymin=-30,ymax=30)
  ax.set_zlim(-30,30)

  lamb*=1.2
  t+=1

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=200, blit=False, repeat=True)
anim

In [ ]:
# Data encrypted by Quantum

fig = plt.figure()
ax = plt.axes(projection='3d')

lamb = 1E-10
dsdt = [0,0,0]
a = 0
t = 0
u = 0.2

xyz = np.linspace(-1,1,Nbody)

#animation.embed_limit = 22076560

def frame(w):
  ax.clear()
  global lamb
  global dsdt
  global u
  global a
  global t

  A = 1*np.cos(2*np.pi/lamb * xyz)

  B = 1*np.sin(2*np.pi/lamb * xyz)

  C = 1*np.tan(2*np.pi/lamb * xyz)

  D = 0.5*np.arctan(2*np.pi/lamb * xyz) # gravity candidate

  points = np.array(fibonacci_sphere(Nbody,C,21,21,20)) * (np.array([D,D,D]).T)

  dsdt[0] += (u+a*t) # x velocity
  dsdt[1] += 0 # y velocity
  dsdt[2] += 0 # z velocity

  plot = ax.scatter3D(np.transpose(points)[0]+dsdt[0],np.transpose(points)[1]+dsdt[1],np.transpose(points)[2]+dsdt[2])
  plot = ax.plot3D(np.transpose(points)[0]+dsdt[0],np.transpose(points)[1]+dsdt[1],np.transpose(points)[2]+dsdt[2],color='b',alpha=0.4)
  ax.axis(xmin=-30,xmax=30)
  ax.axis(ymin=-30,ymax=30)
  ax.set_zlim(-30,30)

  lamb*=1.2
  t+=1

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=200, blit=False, repeat=True)
anim

In [ ]:
# Data encrypted by Quantum

fig = plt.figure()
ax = plt.axes(projection='3d')

lamb = 1E-10
dsdt = [0,0,0]
a = 0
t = 0
u = 0.2

xyz = np.linspace(-1,1,Nbody)

#animation.embed_limit = 22076560

def frame(w):
  ax.clear()
  global lamb
  global dsdt
  global u
  global a
  global t

  A = 1*np.cos(2*np.pi/lamb * xyz)

  B = 1*np.sin(2*np.pi/lamb * xyz)

  C = 1*np.tan(2*np.pi/lamb * xyz)

  D = 0.5*np.arctan(2*np.pi/lamb * xyz) # gravity candidate

  points = np.array(fibonacci_sphere(Nbody,A,21,21,20)) * (np.array([D,D,A]).T)

  dsdt[0] += (u+a*t) # x velocity
  dsdt[1] += 0 # y velocity
  dsdt[2] += 0 # z velocity

  plot = ax.scatter3D(np.transpose(points)[0]+dsdt[0],np.transpose(points)[1]+dsdt[1],np.transpose(points)[2]+dsdt[2])
  plot = ax.plot3D(np.transpose(points)[0]+dsdt[0],np.transpose(points)[1]+dsdt[1],np.transpose(points)[2]+dsdt[2],color='b',alpha=0.4)
  #ax.axis(xmin=-50,xmax=50)
  #ax.axis(ymin=-50,ymax=50)
  #ax.set_zlim(-30,30)

  lamb*=1.2
  t+=1

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=200, blit=False, repeat=True)
anim

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

lamb = 1E-10
xyz = np.linspace(-1,1,Nbody)

def frame(w):
  ax.clear()
  global lamb

  A = 20*np.cos(2*np.pi/lamb * xyz)

  B = 10*np.cos(2*np.pi/lamb * xyz)

  points = fibonacci_sphere(Nbody,A,20,20,20) + fibonacci_sphere(Nbody,B,20,20,20) + fibonacci_sphere(Nbody,B,80,80,80)

  plot = ax.scatter3D(np.transpose(points)[0],np.transpose(points)[1],np.transpose(points)[2])

  lamb*=0.25E1

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=40, blit=False, repeat=True)
anim

## Machine Learning Encryption

In [ ]:
def order_gen(xmin,xmax, xpoints,ymin,ymax,ypoints,zmin,zmax,zpoints):
    
    x_ = np.linspace(xmin, xmax, xpoints)
    y_ = np.linspace(ymin, ymax, ypoints)
    z_ = np.linspace(zmin, zmax, zpoints)

    x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')

    order_xMatrix = np.vstack([x[:,0,0]]*(T+1))
    order_yMatrix = np.vstack([y[:,0,0]]*(T+1))
    order_zMatrix = np.vstack([z[:,0,0]]*(T+1))

    order_xtrained = np.transpose(order_xMatrix) + np.array(xtrained)[:,:,0] - 20
    order_ytrained = np.transpose(order_yMatrix) + np.array(ytrained)[:,:,0] - 20
    order_ztrained = np.transpose(order_zMatrix) + np.array(ztrained)[:,:,0] - 20
    
    return order_xMatrix, order_yMatrix, order_zMatrix 

In [ ]:
def generator3D(xdata, ydata, zdata, order):
    
  order_xMatrix, order_yMatrix, order_zMatrix  = order_gen(min(xdata),max(xdata),len(xtrained),
                                                           min(ydata),max(ydata),len(ytrained),
                                                           min(zdata),max(zdata),len(ztrained))

  
  amount = len(xdata)
  most_random = 0*np.ones([amount,21])
  distance_rand = ((most_random - np.transpose(order_xMatrix))**2 + (most_random - np.transpose(order_yMatrix))**2+ (most_random - np.transpose(order_zMatrix))**2)**0.5
  distance_sum_rand = np.sum(distance_rand**2, axis = 0)
  order_amount = (distance_sum_rand / (Nbody**0.5 *((max(xdata)-min(xdata))**2+(max(ydata)-min(ydata))**2+(max(zdata)-min(zdata))**2)))**0.5  # amount of order
  most_random_factor = np.round(order_amount[0],3)
  
  distance_sum = amount*order*most_random_factor*((max(xdata)-min(xdata))**2+(max(ydata)-min(ydata))**2+(max(zdata)-min(zdata))**2)**0.5  # amount of order
  
  av_distance = distance_sum/amount

  distance = av_distance*np.random.rand(amount)
  
  theta = 2*np.pi *np.random.rand(amount)
  phi = 2*np.pi *np.random.rand(amount) - np.pi

  # spherical coordinates
  xdist = abs(distance*abs(np.transpose(np.cos(theta)*np.sin(phi))))
  ydist = abs(distance*abs(np.transpose(np.sin(theta)*np.sin(phi))))
  zdist = abs(distance*abs(np.transpose(np.cos(phi))))

  genx = (most_random_factor - order*most_random_factor)*(xdata + xdist)/most_random_factor
  geny = (most_random_factor - order*most_random_factor)*(ydata + ydist)/most_random_factor
  genz = (most_random_factor - order*most_random_factor)*(zdata + zdist)/most_random_factor
  
  return genx, geny, genz

In [ ]:
# Data

Nbody = len(xtrained)
lamb = 1E10
xyz = np.linspace(-1,1,Nbody)

A = 20*np.cos(2*np.pi/lamb * xyz)
B = 10*np.cos(2*np.pi/lamb * xyz)

points = fibonacci_sphere(round(Nbody/2),A,20,20,20) + fibonacci_sphere(round(Nbody/2),B,20,20,20)

x_ordered_sphere = np.transpose(points)[0]
y_ordered_sphere = np.transpose(points)[1]
z_ordered_sphere = np.transpose(points)[2]

In [ ]:
# Data encrypted by machine learning

fig = plt.figure()
ax = plt.axes(projection='3d')

order = 1

def frame(w):
  ax.clear()
  global order

  xg, yg, zg = generator3D(x_ordered_sphere,y_ordered_sphere,z_ordered_sphere,order)

  plot = ax.scatter3D(xg,yg,zg)
  ax.set_xlim(0,40)
  ax.set_ylim(0,40)
  ax.set_zlim(0,40)

  order-=0.01

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=101, blit=False, repeat=True)
anim

In [ ]:
# Data encrypted by machine learning

fig = plt.figure()
ax = plt.axes(projection='3d')

order = 1

def frame(w):
  ax.clear()
  global order

  xg, yg, zg = generator3D(x_ordered_sphere,y_ordered_sphere,z_ordered_sphere,order)

  plot = ax.scatter3D(xg,yg,zg)
  ax.set_xlim(0,40)
  ax.set_ylim(0,40)
  ax.set_zlim(0,40)

  order-=0.01

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=101, blit=False, repeat=True)
anim

Quantum Machine Secure Encoding

In [ ]:
# Data

Nbody = len(xtrained)
lamb = 1E10
xyz = np.linspace(-1,1,Nbody)

A = 20*np.cos(2*np.pi/lamb * xyz)
B = 10*np.cos(2*np.pi/lamb * xyz)

points = fibonacci_sphere(round(Nbody/2),A,20,20,20) + fibonacci_sphere(round(Nbody/2),B,20,20,20)

x_ordered_sphere = np.transpose(points)[0]
y_ordered_sphere = np.transpose(points)[1]
z_ordered_sphere = np.transpose(points)[2]

In [ ]:
# Data encrypted by Quantum

fig = plt.figure()
ax = plt.axes(projection='3d')

lamb = 1E-10
xyz = np.linspace(-1,1,Nbody)
order = 1

def frame(w):
  ax.clear()
  global lamb
  global order

  A = 20*np.cos(2*np.pi/lamb * xyz)
  B = 10*np.cos(2*np.pi/lamb * xyz)

  

  points = A # quantum

  xg, yg, zg = generator3D(x_ordered_sphere,y_ordered_sphere,z_ordered_sphere,order) # machine learning

  plot = ax.scatter3D(xg+np.transpose(points)[0],yg+np.transpose(points)[1],zg+np.transpose(points)[2])
  ax.set_xlim(0,75)
  ax.set_ylim(0,75)
  ax.set_zlim(0,75)

  lamb = 1E-5
  order-=0.01

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=200, blit=False, repeat=True)
anim

Quantum Machine Network Transfer

In [ ]:
def generator3D(xdata, ydata, zdata, order):
    
  order_xMatrix, order_yMatrix, order_zMatrix  = order_gen(min(xdata),max(xdata),len(xtrained),
                                                           min(ydata),max(ydata),len(ytrained),
                                                           min(zdata),max(zdata),len(ztrained))

  
  amount = len(xdata)
  most_random = 100*np.ones([amount,21])
  distance_rand = ((most_random - np.transpose(order_xMatrix))**2 + (most_random - np.transpose(order_yMatrix))**2+ (most_random - np.transpose(order_zMatrix))**2)**0.5
  distance_sum_rand = np.sum(distance_rand**2, axis = 0)
  order_amount = (distance_sum_rand / (Nbody**0.5 *((max(xdata)-min(xdata))**2+(max(ydata)-min(ydata))**2+(max(zdata)-min(zdata))**2)))**0.5  # amount of order
  most_random_factor = np.round(order_amount[0],3)
  
  distance_sum = amount*order*most_random_factor*((max(xdata)-min(xdata))**2+(max(ydata)-min(ydata))**2+(max(zdata)-min(zdata))**2)**0.5  # amount of order
  
  av_distance = distance_sum/amount

  distance = av_distance*np.random.rand(amount)
  
  theta = 2*np.pi *np.random.rand(amount)
  phi = 2*np.pi *np.random.rand(amount) - np.pi

  # spherical coordinates
  xdist = abs(distance*abs(np.transpose(np.cos(theta)*np.sin(phi))))
  ydist = abs(distance*abs(np.transpose(np.sin(theta)*np.sin(phi))))
  zdist = abs(distance*abs(np.transpose(np.cos(phi))))

  genx = (most_random_factor - order*most_random_factor)*(xdata + xdist)/most_random_factor
  geny = (most_random_factor - order*most_random_factor)*(ydata + ydist)/most_random_factor
  genz = (most_random_factor - order*most_random_factor)*(zdata + zdist)/most_random_factor
  
  return genx, geny, genz  #, [distance, theta, phi]

In [ ]:
# Data

Nbody = len(xtrained)
lamb = 1E10
xyz = np.linspace(-1,1,Nbody)

A = 20*np.cos(2*np.pi/lamb * xyz)
B = 10*np.cos(2*np.pi/lamb * xyz)

points = fibonacci_sphere(round(Nbody/3+1),A,20,20,20) + fibonacci_sphere(round(Nbody/3),B,150,50,50) + fibonacci_sphere(round(Nbody/3),A,20,130,80)

x_ordered_sphere = np.transpose(points)[0]
y_ordered_sphere = np.transpose(points)[1]
z_ordered_sphere = np.transpose(points)[2]

In [ ]:
# Data encrypted by Quantum

fig = plt.figure()
ax = plt.axes(projection='3d')

lamb = 1E-10
xyz = np.linspace(-1,1,Nbody)
order = 1

def frame(w):
  ax.clear()
  global lamb
  global order

  A = 10*np.cos(2*np.pi/lamb * xyz)
  B = 20*np.cos(2*np.pi/lamb * xyz +90)

  

  points = fibonacci_sphere(round(Nbody),A,500,500,500)

  xg, yg, zg = generator3D(x_ordered_sphere,y_ordered_sphere,z_ordered_sphere,order)  # machine learning

  plot = ax.scatter3D(xg,yg,zg)
  #ax.set_xlim(0,1000)
  #ax.set_ylim(0,1000)
  #ax.set_zlim(0,1000)

  lamb = 1E-5
  order-=0.01

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=200, blit=False, repeat=True)
anim

In [ ]:
# Data encrypted by Quantum

fig = plt.figure()
ax = plt.axes(projection='3d')

lamb = 1E-10
xyz = np.linspace(-1,1,Nbody)
order = 1

def frame(w):
  ax.clear()
  global lamb
  global order

  A = 10*np.cos(2*np.pi/lamb * xyz)
  B = 20*np.cos(2*np.pi/lamb * xyz +90)

  

  points = fibonacci_sphere(round(Nbody),A,500,500,500)

  xg, yg, zg = generator3D(x_ordered_sphere,y_ordered_sphere,z_ordered_sphere,order)  # machine learning

  plot = ax.scatter3D(xg,yg,zg)
  #ax.set_xlim(0,1000)
  #ax.set_ylim(0,1000)
  #ax.set_zlim(0,1000)

  lamb*=1.2
  order-=0.01

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=200, blit=False, repeat=True)
anim

# Image Encryption machine learning

In [ ]:
import cv2
from matplotlib import pyplot as plt

im_rgb = cv2.imread("Images/World-Map-Board_Original-Map.jpg")

scale_percent = 304 / len(im_rgb[0])*len(im_rgb) / 100 # percent of original size
width = int(im_rgb.shape[1] * scale_percent / 100)
height = int(im_rgb.shape[0] * scale_percent / 100)
dim = (width, height)

im_rgb = cv2.resize(im_rgb, dim, interpolation = cv2.INTER_AREA)

im_rgb = cv2.cvtColor(im_rgb, cv2.COLOR_BGR2RGB)



plt.imshow(im_rgb)
plt.show()

In [ ]:
len(im_rgb[0])*len(im_rgb)

In [ ]:
coordinates = np.argwhere(im_rgb)

In [ ]:
order = 0

xg, yg, zg = generator3D(coordinates[:,0],coordinates[:,1],coordinates[:,2],order)

fig = plt.figure()
ax = plt.axes(projection='3d')

ax.scatter3D(xg,yg,zg)

In [ ]:
# Data encrypted by machine learning

order = 1

xg2, yg2, zg2 = generator3D(coordinates[:,0],coordinates[:,1],coordinates[:,2],order)

fig = plt.figure()
ax = plt.axes(projection='3d')

ax.scatter3D(xg2,yg2,zg2)

In [ ]:
    # Data encrypted by machine learning

fig = plt.figure()
ax = plt.axes(projection='3d')

order = 1

def frame(w):
  ax.clear()
  global order

  xg, yg, zg = generator3D(coordinates[:,0],coordinates[:,1],coordinates[:,2],order)

  plot = ax.scatter3D(xg,yg,zg)

  order-=0.1

  return plot

plt.close()

anim = animation.FuncAnimation(fig, frame, frames=10, blit=False, repeat=True)
anim

In [ ]:


# Convert back to PIL Image and re-apply original palette
r = Image.fromarray([xg2,yg2,zg2],mode='P') 
r.putpalette(im.getpalette())

# Optionally save
r.save('result.png')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Simulation parameters
v0           = 3      # velocity
eta          = 360     # random fluctuation in angle (in radians)
Lx           = 70        # size of box
Ly           = 40
R            = 0.89      # interaction radius
dt           = 0.1      # time step
Nt           = 200      # number of time steps
N            = 304      # number of birds
plotRealTime = True

# Initialize
#np.random.seed(30)      # set the random number generator seed

# bird positions
x = np.random.rand(N,1)*Lx
y = np.random.rand(N,1)*Ly

x = np.array(ordered_orig['Center X']).reshape(304,1)
y = np.array(ordered_orig['Center Y']).reshape(304,1)


# bird velocities
theta = 2 * np.pi * np.random.rand(N,1)
vx = v0 * np.cos(theta)
vy = v0 * np.sin(theta)

# Prep figure
fig = plt.figure(figsize=(6,6), dpi=96)
ax = plt.gca()

# Simulation Main Loop
for i in range(Nt):

    # move
    x = x + vx*dt
    y = y + vy*dt
    
    # apply periodic BCs
    x = x % Lx
    y = y % Ly
    
    # find mean angle of neighbors within R
    mean_theta = theta
    for b in range(N):
        neighbors = (x-x[b])**2+(y-y[b])**2 < R**2
        sx = np.sum(np.cos(theta[neighbors]))
        sy = np.sum(np.sin(theta[neighbors]))
        mean_theta[b] = np.arctan2(sy, sx)
    
    # add random perturbations
    theta = mean_theta + eta*(np.random.rand(N,1)-0.5)
    
    # update velocities
    vx = v0 * np.cos(theta)
    vy = v0 * np.sin(theta)
    
    # plot in real time
    if plotRealTime or (i == Nt-1):
        plt.cla()
        plt.scatter(x,y,s=0.8393, facecolors='none', edgecolors='r')
        ax.set(xlim=(0, Lx), ylim=(0, Ly))
        ax.set_aspect('equal')
        plt.pause(0.001)
        clear_output(wait=True)
# Save figure
plt.show()